In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

In [ ]:
# Get current path: all relative paths that you may use for input/output start from here.
#
# - Colab:  The default path is /content, and your GDrive folder is accessible (if mounted) at /content/drive/MyDrive
#
# - local:  If running on your local machine, current_path is the position WHERE YOU LAUNCHED THE NOTEBOOK KERNEL BY 'jupyter notebook'
#           and NOT the directory in which this ipynb file is saved.
#           If you want to modify your current path, go to the terminal, stop the kernel (ctrl-C + press y), then move to the desired path by
#           cd my/desired/path/starting/from/here
#           and then launch 'jupyter notebook'.
import os
current_path = os.getcwd()
print(current_path)

---
---
# Exercise 1
## Solve nonlinear, steady Navier-Stokes problem.

\begin{equation*}
\begin{cases}
(\boldsymbol{u}\cdot\nabla)\boldsymbol{u} - \nu\Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega, \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, \\
\boldsymbol{u} = 4y(1-y)\boldsymbol{i} & {\rm on} \ \Gamma_{\rm in}, \\
(\nu\nabla\boldsymbol{u} - pI)\boldsymbol{n} = \boldsymbol{0} & {\rm on} \ \Gamma_{\rm out}, \\
\boldsymbol{u} = \boldsymbol{0} & {\rm on} \ \Gamma_{\rm wall}=\partial\Omega\setminus(\Gamma_{\rm in}\cup\Gamma_{\rm out}). \\
\end{cases}
\end{equation*}

In [ ]:
# Import mesh: set the path correctly!
# See cell before Exercise 1 about current path.
mesh1='/content/drive/MyDrive/Cfd/elbow1.msh'
mesh2='/content/drive/MyDrive/Cfd/elbow2.msh'


mesh = Mesh(mesh1)
fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend(loc='upper left')

In [ ]:
# Function spaces (mixed formulation)


V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])


# Data and boundary conditions
nu = 0.01
f = Constant((0.,0.))
x = SpatialCoordinate(mesh)

u_in = (4*x[1]*(1-x[1]), 0.)

bc_wall = DirichletBC(W.sub(0),Constant((0.0,0.0)) , (3))

bc_in=DirichletBC(W.sub(0),u_in, (1))




bcs = (bc_wall, bc_in)

### Variational problems

In [ ]:
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

def initialization_problem_forms(u, v, p, q, nu, f):
    # Stokes initialization
    a_stokes = nu*inner(grad(u),grad(v))*dx -p*div(v)*dx +q*div(u)*dx
    L_stokes = inner(f,v)*dx
    return a_stokes, L_stokes

def nonlinear_iteration_forms(u, v, p, q, nu, f, u_old):
    # Fixed-point iteration
    a_fp = nu*inner(grad(u),grad(v))*dx -p*div(v)*dx +q*div(u)*dx +dot(dot(grad(u),u_old),v)*dx  #dot for matrix vec multi
    L_fp = inner(f,v)*dx
    return a_fp, L_fp

### Initialization and definition of the ***linear*** solver for each nonlinear iteration.

In [ ]:
from firedrake.mesh import temp_internal_comm
# Initialization solver
wh_init = Function(W)
a_init, L_init = initialization_problem_forms(u, v, p, q, nu, f)
init_problem = LinearVariationalProblem(a_init,L_init,wh_init,bcs)

param = {'ksp_type': 'gmres', 'pc_type': 'bjacobi', 'sub_pc_type': 'ilu'}
solver_init =  LinearVariationalSolver(init_problem,solver_parameters=param )

# Actual initialization
solver_init.solve()

uh, ph = wh_init.split()

# Plot of initial guess (Stokes solution)
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

# vtk output for Paraview
outfileU = File("output/velocity.pvd")
outfileP = File("output/pressure.pvd")
uh.rename("Velocity")   # this name will be used in Paraview
ph.rename("Pressure")   # this name will be used in Paraview
outfileU.write(uh)
outfileP.write(ph)

# Nonlinear solver
u_old = Function(V)
u_old.assign(uh)  #real copy
#different from u_old=u_h shallow copy

p_old = Function(Q)
p_old.assign(ph)

temp_u=Function(V)
temp_u.assign(uh)

temp_p=Function(Q)
temp_p.assign(ph)



In [ ]:
a, L = nonlinear_iteration_forms(u, v, p, q, nu, f, u_old)
...
problem = LinearVariationalProblem(a,L,wh_init,bcs)

param = {'ksp_type': 'gmres', 'pc_type': 'bjacobi', 'sub_pc_type': 'ilu'}
solver =  LinearVariationalSolver(problem,solver_parameters=param )

### Iterative algorithm for the solution of the nonlinear problem

In [ ]:
maxit = 100
it = 0
tol = 1e-3
err = tol+1     # >tol in order to enter the loop at the beginning


u_old=temp_u
p_old=temp_p

while it <= maxit and err > tol:

    it += 1



    a, L = nonlinear_iteration_forms(u, v, p, q, nu, f, u_old)
    wh = Function(W)

    problem = LinearVariationalProblem(a,L,wh,bcs)

    param = {'ksp_type': 'gmres', 'pc_type': 'bjacobi', 'sub_pc_type': 'ilu'}
    solver =  LinearVariationalSolver(problem,solver_parameters=param )
    solver.solve()

    uh, ph = wh.split()



    err =errornorm(uh,u_old,'H1',mesh=mesh)/norm(u_old,'H1',mesh=mesh) + errornorm(ph,p_old,'L2',mesh=mesh)/norm(p_old,'L2',mesh=mesh)


    print("Iteration = ", it, " Error = ", err)
    uh.rename("Velocity")
    ph.rename("Pressure")
    outfileU.write(uh)
    outfileP.write(ph)

    u_old.assign(uh) # update the old solution
    p_old.assign(ph) # update the old solution

if it <= maxit:
    print('Nonlinear solver converged in', it, 'iterations.')
else:
    print('Nonlinear solver di NOT converge!\nRelative error =', err, 'after', it, 'iterations.')

In [ ]:
uh, ph = wh_init.subfunctions

# Plot of initial guess (Stokes solution)
fig, ax = plt.subplots(2,2,figsize=(10, 10))



col = tripcolor(ph, axes=ax[0,0])
plt.colorbar(col)
ax[0,0].title.set_text(' Ns \n\n \
pressure')

col = quiver(uh, axes=ax[1,0])
plt.colorbar(col)
ax[1,0].title.set_text('velocity')

col = tripcolor(temp_p, axes=ax[0,1])
plt.colorbar(col)
ax[0,1].title.set_text(' Stokes \n\n \
pressure')

col = quiver(temp_u, axes=ax[1,1])
plt.colorbar(col)
ax[1,1].title.set_text('velocity')


fig.subplots_adjust(hspace=0.5, bottom=0.1)


---
---
# Exercise 2
## Solve nonlinear, steady Navier-Stokes and compute streamlines.

The differences with respect to Exercise 1 are in the mesh, viscosity, and boundary conditions.

In [ ]:
# Import mesh: set the path correctly!
# See cell before Exercise 1 about current path.
mesh = ...
fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend(loc='upper left')

# Function spaces
...

# Data and boundary conditions
...
bcs = ...

Then, the nonlinear problem and its iterative solution algorithm are the same as in Exercise 1: after the execution of the cell just above these, run all the cells of Exercise 1 starting with 'Variational problems'.



Here we implement just the stream function problem and solution.

In [ ]:
Z = ...
psi = TrialFunction(Z)
eta = TestFunction(Z)

a_stream = ...
# omega = Dx(uh.sub(1),0) - Dx(uh.sub(0),1)   # by components (UFL syntax)
omega = rot(uh)                               # by pre-defined UFL command
L_stream = ...

bcs_stream = ...

psi_h = Function(Z)
... solve stream problem ...

col = tricontour(psi_h)     # contour plot with no filling
plt.colorbar(col)
col = tricontourf(psi_h)    # contour plot with filled-in coloring
plt.colorbar(col)